In [227]:
import numpy as np # linear algebra
import pandas as pd # CSV file I/O (e.g. pd.read_csv)
from datetime import datetime
import calendar
import warnings
from math import pi, sin, cos, sqrt, atan2, radians,asin
import calendar
import os # reading the input files we have access to
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

In [332]:
train=pd.read_csv("input/train.csv",nrows=1000000) #Has to be 10^6 for good performance [TODO]
print("Shape of Training Data",train.shape)
test=pd.read_csv("input/test.csv")
print("Shape of Testing Data", test.shape)

Shape of Training Data (1000000, 8)
Shape of Testing Data (9914, 7)


In [333]:
def encodeDays(day_of_week):
    day_dict={'Sunday':0,'Monday':1,'Tuesday':2,'Wednesday':3,'Thursday':4,'Friday':5,'Saturday':6}
    return day_dict[day_of_week]
def clean_data(data):
    boundary={'min_lng':-74.263242,
              'min_lat':40.573143,
              'max_lng':-72.986532, 
              'max_lat':41.709555}
    
    data['pickup_datetime']=pd.to_datetime(data['pickup_datetime'],format='%Y-%m-%d %H:%M:%S UTC')
    data['pickup_day']=data['pickup_datetime'].apply(lambda x:x.day)
    #data['pickup_sin_hm'] = data['pickup_datetime'].apply(lambda x:sin(2*pi*(x.hour * 60 + x.minute)/1440))
    data['pickup_hour']=data['pickup_datetime'].apply(lambda x:x.hour)
    data['pickup_month']=data['pickup_datetime'].apply(lambda x:x.month)
    data['pickup_year']=data['pickup_datetime'].apply(lambda x:x.year)
    if 'fare_amount' in data.columns:
        data=data[data['fare_amount']>=0]
        data.loc[~((data.pickup_longitude >= boundary['min_lng'] ) & (data.pickup_longitude <= boundary['max_lng']) &
            (data.pickup_latitude >= boundary['min_lat']) & (data.pickup_latitude <= boundary['max_lat']) &
            (data.dropoff_longitude >= boundary['min_lng']) & (data.dropoff_longitude <= boundary['max_lng']) &
            (data.dropoff_latitude >=boundary['min_lat']) & (data.dropoff_latitude <= boundary['max_lat'])),'is_outlier_loc']=1
        data.loc[((data.pickup_longitude >= boundary['min_lng'] ) & (data.pickup_longitude <= boundary['max_lng']) &
            (data.pickup_latitude >= boundary['min_lat']) & (data.pickup_latitude <= boundary['max_lat']) &
            (data.dropoff_longitude >= boundary['min_lng']) & (data.dropoff_longitude <= boundary['max_lng']) &
            (data.dropoff_latitude >=boundary['min_lat']) & (data.dropoff_latitude <= boundary['max_lat'])),'is_outlier_loc']=0

    #print("Outlier vs Non Outlier Counts")
    #print(data['is_outlier_loc'].value_counts())

    # Let us drop rows, where location is outlier
        data=data.loc[data['is_outlier_loc']==0]
        data.drop(['is_outlier_loc'],axis=1,inplace=True)
    
    data=data[data['passenger_count']<=8]
    
    weekday_names = {0: 'Monday',
                 1: 'Tuesday',
                 2: 'Wednesday',
                 3: 'Thursday',
                 4: 'Friday',
                 5: 'Saturday',
                 6: 'Sunday'}
    #data = data.join(pd.get_dummies("is"+data['pickup_datetime'].dt.weekday.apply(lambda x: weekday_names.get(x))))
    #data = data.join(pd.get_dummies("is"+data['pickup_datetime'].dt.month))
    data['pickup_day_of_week']=data['pickup_datetime'].apply(lambda x:calendar.day_name[x.weekday()])
    data['pickup_day_of_week'] = data['pickup_day_of_week'].apply(lambda x: encodeDays(x))
    #^one-hot encoding of weekday
    return data

In [334]:
train=clean_data(train)
test=clean_data(test)
print("Shape of Training Data after cleaning ",train.shape)
print("Shape of Testing Data after cleaning", test.shape)

/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


Shape of Training Data after cleaning  (978799, 13)
Shape of Testing Data after cleaning (9914, 12)


In [335]:
train.describe

<bound method NDFrame.describe of                                   key  fare_amount     pickup_datetime  \
0         2009-06-15 17:26:21.0000001         4.50 2009-06-15 17:26:21   
1         2010-01-05 16:52:16.0000002        16.90 2010-01-05 16:52:16   
2        2011-08-18 00:35:00.00000049         5.70 2011-08-18 00:35:00   
3         2012-04-21 04:30:42.0000001         7.70 2012-04-21 04:30:42   
4       2010-03-09 07:51:00.000000135         5.30 2010-03-09 07:51:00   
5         2011-01-06 09:50:45.0000002        12.10 2011-01-06 09:50:45   
6         2012-11-20 20:35:00.0000001         7.50 2012-11-20 20:35:00   
7        2012-01-04 17:22:00.00000081        16.50 2012-01-04 17:22:00   
8       2012-12-03 13:10:00.000000125         9.00 2012-12-03 13:10:00   
9        2009-09-02 01:11:00.00000083         8.90 2009-09-02 01:11:00   
10        2012-04-08 07:30:50.0000002         5.30 2012-04-08 07:30:50   
12        2009-11-06 01:04:03.0000002         4.10 2009-11-06 01:04:03   
13  

In [336]:
def processDataForModelling(data,target,drop_cols,is_train=True,split=0.25):
    data_1=data.drop(drop_cols,axis=1)
    if is_train==True:
        X=data_1.drop([target],axis=1)
        y=data_1[target]
        X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=split,random_state=123)
        
        print("Shape of Training Features",X_train.shape)
        print("Shape of Validation Features ",X_test.shape)
        
        return X_train, X_test, y_train, y_test
    else:
        print ("Shape of Test Data",data_1.shape)
        return data_1

In [337]:
X_train, X_test, y_train, y_test=processDataForModelling(train,'fare_amount',drop_cols=['key','pickup_datetime'],is_train=True,split=0.2)

Shape of Training Features (783039, 10)
Shape of Validation Features  (195760, 10)


In [338]:
X_train.describe

<bound method NDFrame.describe of         pickup_longitude  pickup_latitude  dropoff_longitude  \
975541        -73.870807        40.773718         -73.987162   
874090        -73.998912        40.719423         -73.994190   
543271        -73.994255        40.730018         -73.994583   
412003        -74.002666        40.726937         -73.982204   
401094        -73.983312        40.754823         -73.946593   
351836        -73.985810        40.740675         -73.983197   
812933        -73.963642        40.774420         -73.963893   
293926        -73.974355        40.759955         -73.955708   
4232          -73.989004        40.730487         -73.991177   
599955        -73.981979        40.773564         -73.981548   
287151        -74.005470        40.739407         -74.001587   
668764        -73.990278        40.752050         -73.996807   
211805        -74.002368        40.733675         -73.999120   
2980          -73.966778        40.753462         -73.976775   
228737

In [339]:
test_data=processDataForModelling(test,'fare_amount',drop_cols=['key','pickup_datetime'],is_train=False)

Shape of Test Data (9914, 10)


In [340]:
avg_fare=round(np.mean(y_train),2)
baseline_pred=np.repeat(avg_fare,y_test.shape[0])
baseline_rmse=np.sqrt(mean_squared_error(baseline_pred, y_test))
print("Basline RMSE of Validation data :",baseline_rmse)

Basline RMSE of Validation data : 9.659505711183623


In [341]:
lm = LinearRegression()
lm.fit(X_train,y_train)
y_pred=np.round(lm.predict(X_test),2)
lm_rmse=np.sqrt(mean_squared_error(y_pred, y_test))
print("RMSE for Linear Regression is ",lm_rmse)

RMSE for Linear Regression is  8.219077313138138


In [342]:
rf = RandomForestRegressor(n_estimators = 100, random_state = 1,n_jobs=-1, verbose=2)
rf.fit(X_train,y_train)
rf_pred= rf.predict(X_test)
rf_rmse=np.sqrt(mean_squared_error(rf_pred, y_test))
print("RMSE for Random Forest is ",rf_rmse)
print(rf.feature_importances_)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


building tree 1 of 100building tree 2 of 100building tree 3 of 100
building tree 4 of 100


building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100building tree 23 of 100

building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  2.4min


building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72 of 100
building tree 73 of 100
building tree 74 of 100
building tree 75 of 100
building tree 76 of 100
building tree 77 of 100
building tree 78

[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  6.9min finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    4.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:   11.7s finished


RMSE for Random Forest is  3.9809285947728537
[0.29623826 0.12151069 0.37385929 0.133674   0.00401601 0.01286484
 0.01483161 0.01041534 0.02494751 0.00764245]
